In [7]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
runtime = pd.read_parquet("../raw-data/runtimeSeveralRoutesOctWithLags_all.gzip")

In [27]:
runtime.routeId.unique()

['17', '33', '48', '47', '21', ..., '60', '79', 'G', '18', '6']
Length: 13
Categories (13, object): ['17', '33', '48', '47', ..., '79', 'G', '18', '6']

In [8]:
# Select 
query = "(routeId.isin(['7', '21', '47', '33'])) & (serviceDate == '2022-10-26')"
runtime_sel = runtime.query(query).copy()

In [20]:
runtime_sel.routeId.unique()

['33', '47', '21']
Categories (13, object): ['17', '33', '48', '47', ..., '79', 'G', '18', '6']

In [11]:
# All the trips on this day with their time range
trips = runtime_sel.groupby('tripId')["observedToStopArrivalTime"].agg(["min", "max"])

In [12]:
trips.head()

,min,max
tripId,,
60622,2022-10-26 20:40:16,2022-10-26 21:08:17
60623,2022-10-26 17:18:00,2022-10-26 17:58:05
60624,2022-10-26 17:23:36,2022-10-26 18:04:23
60625,2022-10-26 17:36:18,2022-10-26 18:18:09
60626,2022-10-26 17:50:34,2022-10-26 18:29:53


In [17]:
desired_time = "2022-10-26 08:30:16"

In [14]:
len(trips)

586

In [19]:
len(trips.query("min < @desired_time < max"))

26